# Plotting animations of land cover <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the `DEA Sandbox` environment
* **Products used:** 
[ga_ls_landcover_class_cyear_2](https://explorer.sandbox.dea.ga.gov.au/products/ga_ls_landcover_class_cyear_2)
* This is a modified version of the DEA [notebook](https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Frequently_used_code/Land_cover_animated_plots.ipynb)


## Background

Land cover is the physical surface of the Earth, including trees, shrubs, grasses, soils, exposed rocks, water bodies, plantations, crops and built structures.
Digital Earth Australia Land Cover (DEA Land Cover) is a continental dataset that maps annual land cover classifications for Australia from 1988 to the present. 
Detailed information about DEA Land Cover can be found in the [DEA Land Cover notebook](../DEA_datasets/DEA_Land_Cover.ipynb) and on the [DEA Land Cover product details](https://cmi.ga.gov.au/data-products/dea/607/dea-land-cover-landsat#details) page.

This modified version is used to look at land use change in catchments in the south east of South Australia.

### Load packages

In [1]:
%matplotlib inline

import math
import os
import sys

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from xrspatial.classify import reclassify #in a terminal (add a new tab, blue plus button):  pip install xarray-spatial

sys.path.insert(1, os.path.abspath("../Tools"))
from dea_tools.landcover import lc_animation, plot_land_cover
from dea_tools.plotting import display_map
from dea_tools.spatial import xr_rasterize
from dea_tools.plotting import rgb, map_shapefile

from datacube.utils.cog import write_cog

### Connect to the datacube
Connect to the datacube so we can access DEA data. 

In [2]:
dc = datacube.Datacube(app="Land_cover_animated_plots")

### Create query and load a time series of land cover data

In order to generate animations of DEA Land Cover, we first need to load a time series of data for an area. 
As an example, let's load both the base 6-class classification (Level 3) layer and the full classification (Level 4) layer based on a shapefile bounding box.

In [3]:
gdf = gpd.read_file('shp/Model_Domain.shp')
bounds=gpd.GeoDataFrame(gdf).to_crs("EPSG:4326").bounds

#bounds need to be slightly larger for later clip to work
lon=(bounds.minx,bounds.maxx)
lat=(bounds.miny,bounds.maxy)
display_map(x=lon, y=lat)

In [9]:
# Build query and load data

for year in range(1990, 2021, 5):
    query = {
        "y": lat,
        "x": lon,
        "time": (str(year-4), str(year)),
    }

    # Load DEA Land Cover data from the datacube
    land_cover_data = dc.load(
        product="ga_ls_landcover_class_cyear_2",
        output_crs="EPSG:3577",
        #measurements=["level3", "level4", "waterper_wat_cat_l4d_au"],
        measurements=["lifeform","water_state","level3"],
        resolution=(-25, 25),
        **query
    )
    
    woody=land_cover_data.lifeform.median(dim='time')
    woody=reclassify(woody,bins=[0,1,2],new_values=[np.nan,0,1])
    write_cog(geo_im=woody.astype(np.int16),
          fname='5yrMedian/Woody_' + str(year -4) + '-' + str(year) +'.tif',
          overwrite=True)
    
    water=land_cover_data.water_state.median(dim='time')
    write_cog(geo_im=water.astype(np.int16),
          fname='5yrMedian/Water_' + str(year -4) + '-' + str(year) +'.tif',
          overwrite=True)
                      
    artificial=land_cover_data.level3.median(dim='time')
    artificial=reclassify(artificial,bins=[0,111,112,124,215,216,220],new_values=[np.nan,0,0,0,1,1,0]) #looking at Mt Gambier and Millicent, 'natural bare surface' aligns with built up areas as well as artifical surface.
    write_cog(geo_im=artificial.astype(np.int16),
          fname='5yrMedian/Artificial_' + str(year -4) + '-' + str(year) +'.tif',
          overwrite=True)
    